# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import TruncatedSVD 
from lightgbm import LGBMClassifier

import pickle
import bz2

from sqlalchemy import create_engine
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM disaster_response', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
categories = df[categories].sum(axis=0).sort_values(ascending=False).index[:20]

In [24]:

df[df['genre']=='news'][categories].sum(axis=0).sort_values(ascending=False)

related                   10671
aid_related                5860
weather_related            4280
floods                     1747
other_aid                  1563
storm                      1445
medical_help               1415
infrastructure_related     1230
shelter                    1113
other_weather              1052
food                       1007
earthquake                  910
transport                   872
other_infrastructure        866
death                       858
direct_report               852
buildings                   829
medical_products            793
water                       790
request                     604
dtype: int64

In [26]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [28]:
tokens = df[df['related']==1]['message'].apply(tokenize)

In [29]:
tokens

0        [weather, update, -, a, cold, front, from, cub...
1        [is, the, hurricane, over, or, is, it, not, over]
2                   [looking, for, someone, but, no, name]
3        [un, report, leogane, 80-90, destroyed, ., onl...
4        [say, :, west, side, of, haiti, ,, rest, of, t...
6                    [storm, at, sacred, heart, of, jesus]
7        [please, ,, we, need, tent, and, water, ., we,...
9        [i, am, in, croix-des-bouquets, ., we, have, h...
10       [there, 's, nothing, to, eat, and, water, ,, w...
12       [i, am, in, thomassin, number, 32, ,, in, the,...
13       [let, 's, do, it, together, ,, need, food, in,...
14       [more, information, on, the, 4636, number, in,...
15       [a, comitee, in, delmas, 19, ,, rue, (, street...
16       [we, need, food, and, water, in, klecin, 12, ....
19       [i, would, like, to, know, if, the, earthquake...
20       [i, would, like, to, know, if, one, of, the, r...
21              [i, 'm, in, laplaine, ,, i, am, a, victi